<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/techvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

def get_valuation(self, df):
    return sum([self.macd_valuation(df), self.rsi_valuation(df), self.stoch_valuation(df), self.cci_valuation(df)]) + 4

def rsi_valuation(self, df, n=14, threshold=70):
    def calculate_rsi(column):
        delta = column.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=n).mean()
        avg_loss = loss.rolling(window=n).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.iloc[-1]

    rsi_values = df.apply(calculate_rsi, axis=0)
    valuation_status = rsi_values.apply(lambda x: 1 if x > threshold else (-1 if x < 30 else 0)).to_dict()
    return valuation_status['Close']

def macd_valuation(self, df, short_period=12, long_period=26, signal_period=9, threshold=0):
    def calculate_macd_histogram(column):
        short_ema = column.ewm(span=short_period, adjust=False).mean()
        long_ema = column.ewm(span=long_period, adjust=False).mean()
        macd_line = short_ema - long_ema
        signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
        macd_histogram = macd_line - signal_line
        return macd_histogram

    macd_histograms = df.apply(calculate_macd_histogram, axis=0)
    latest_macd_histogram = macd_histograms.iloc[-1]
    valuation_status = latest_macd_histogram.apply(lambda x: 1 if x > threshold else (-1 if x < -threshold else 0)).to_dict()
    return valuation_status['Close']

def stoch_valuation(self, df, n=14, k=3, d=3, k_threshold=70, d_threshold=70):
    def calculate_stoch(df):
        low_min = df['Low'].rolling(window=n).min()
        high_max = df['High'].rolling(window=n).max()
        pct_k = ((df['Close'] - low_min) / (high_max - low_min)) * 100
        pct_d = pct_k.rolling(window=k).mean()
        slow_pct_d = pct_d.rolling(window=d).mean()
        return pct_k.iloc[-1], pct_d.iloc[-1], slow_pct_d.iloc[-1]

    stoch_values = calculate_stoch(df)
    status = 1 if (stoch_values[0] > k_threshold and stoch_values[1] > d_threshold) else (-1 if (stoch_values[0] < 30 and stoch_values[1] < 30) else 0)
    return status

def cci_valuation(self, df, n=20, constant=0.015, cci_threshold=100):
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    mean_deviation = (typical_price - typical_price.rolling(window=n).mean()).abs().rolling(window=n).mean()
    cci = (typical_price - typical_price.rolling(window=n).mean()) / (constant * mean_deviation)
    status = 1 if cci.iloc[-1] > cci_threshold else (-1 if cci.iloc[-1] < -cci_threshold else 0)
    return status